# WHAT IF ANALYSIS

In [1]:
from webbot import Browser
from zipfile import ZipFile
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
global scaler
scaler = MinMaxScaler()
from sklearn.metrics import *
from IPython.core.display import HTML
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
from sklearn.svm import LinearSVR

import sys
import os
import time
import zipfile
import datetime
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

C:\Users\Abhinav\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
def navigate(user='tiwari.abhi@husky.neu.edu', password="\nWIB[Lw"):
    global web
    web = Browser()
    web.go_to("https://freddiemac.embs.com/FLoan/secure/login.php?pagename=download2")
    web.type(user, into='email')
    web.type(password, into='password')
    web.click('Submit Credentials')
    web.click('Yes')
    web.click('Continue')

In [3]:
def get_next_quarter_str(quarter_str='Q22005'):
    try:
        quarter_int = int(quarter_str[1:2])
        
        if quarter_int <= 0:
            print('Quarter cannot be lesser than or equal to zero')
            exit(0)
        
        year = int(quarter_str[2:])
        
    except:
        print('Cannot Parse Quarter/Year to Int Value')
        exit(0)
    
    next_quarter = quarter_int + 1
    
    if next_quarter > 4:
        next_quarter = 1
        year = year + 1
    
    next_quarter_str = 'Q'+ str(next_quarter) + str(year)
    return next_quarter_str

In [4]:
def get_4_quarter_data(first_quarter,second_quarter,third_quarter,fourth_quarter):
    
    quarters = [first_quarter,second_quarter,third_quarter,fourth_quarter]
    
    global file_names
    file_names = []
    
    for q in quarters:
        web.click('historical_data1_{}.zip'.format(q))
        file_names.append('historical_data1_{}.zip'.format(q))
        time.sleep(20)
    
    global download_path
    download_path = ''
    path_list = os.getcwd().split('\\')[:3]

    for item in path_list:
        download_path = download_path + item + '\\'

    download_path = download_path + 'Downloads'

In [5]:
def assure_path_exists(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [6]:
def extract_zipped_files_to_cwd(path_to_downloaded_files):
    global folder_path_all_files
    folder_path_all_files = os.getcwd() + '\\' + 'Extracted Quarterly Files'
    zip_ref = zipfile.ZipFile(path_to_downloaded_files, 'r')
    assure_path_exists(folder_path_all_files)
    zip_ref.extractall(folder_path_all_files)
    zip_ref.close()

In [7]:
def fillNAN_orig(df):
    df['fico'] = df['fico'].fillna(0)
    df['flag_fthb'] = df['flag_fthb'].fillna('X')
    df['cd_msa'] = df['cd_msa'].fillna(0)
    df['mi_pct'] = df['mi_pct'].fillna(0)
    df['cnt_units'] = df['cnt_units'].fillna(0)
    df['occpy_sts'] = df['occpy_sts'].fillna('X')
    df['cltv'] = df['cltv'].fillna(0)
    df['dti'] = df['dti'].fillna(0)
    df['ltv'] = df['ltv'].fillna(0)
    df['channel'] = df['channel'].fillna('X')
    df['ppmt_pnlty'] = df['ppmt_pnlty'].fillna('X')
    df['prop_type'] = df['prop_type'].fillna('XX')
    df['zipcode'] = df['zipcode'].fillna(0)
    df['loan_purpose'] = df['loan_purpose'].fillna('X')
    df['cnt_borr'] = df['cnt_borr'].fillna(0)
    df['flag_sc'] = df['flag_sc'].fillna('N')
    return df

In [8]:
def changedatatype_orig(df):
    # Change the data types for all column
    df[['fico', 'cd_msa', 'mi_pct', 'cnt_borr', 'cnt_units', 'cltv', 'dti', 'orig_upb', 'ltv', 'zipcode',
        'orig_loan_term']] = df[['fico', 'cd_msa', 'mi_pct', 'cnt_borr', 'cnt_units', 'cltv', 'dti', 'orig_upb', 'ltv', 'zipcode','orig_loan_term']].astype('int64')
    df[['flag_sc', 'servicer_name']] = df[['flag_sc', 'servicer_name']].astype('str')
    return df

In [9]:
def get_Origination_Quarter_File(historic_orig_files):
    writeHeader1 = True

    abc = glob.glob(historic_orig_files)
    xyz = glob.glob(folder_path_all_files + "/historical_data1_time_*.txt")
    
    #Get Rid of all the paths that point to the historical_data1_time_files
    for item in xyz:
        abc.remove(item)
    
    for f in abc:
        sample_df = pd.read_csv(f, sep="|",
                                names=['fico', 'dt_first_pi', 'flag_fthb', 'dt_matr', 'cd_msa', "mi_pct",
                                       'cnt_units', 'occpy_sts', 'cltv', 'dti', 'orig_upb', 'ltv', 'int_rt',
                                       'channel', 'ppmt_pnlty', 'prod_type', 'st', 'prop_type', 'zipcode',
                                       'id_loan', 'loan_purpose', 'orig_loan_term', 'cnt_borr', 'seller_name',
                                       'servicer_name', 'flag_sc'], skipinitialspace=True, low_memory=False)
        
        sample_df = fillNAN_orig(sample_df)
        sample_df = changedatatype_orig(sample_df)
        
        sample_df['Year'] = ['19' + x if x == '99' else '20' + x for x in (sample_df['id_loan'].apply(lambda x: x[2:4]))]
        
        #Get historical File name
        file_name = f.split('\\')[-1].split('.')[0]
        
        sample_df.to_csv(file_name + '.csv', header=True, index=False)

In [10]:
def checkAllReqColumns(df):
    cols_to_keep = ['fico','flag_fthbN','flag_fthbX','flag_fthbY','mi_pct','cnt_units','occpy_stsl','occpy_stsO','occpy_stsS','cltv','dti','orig_upb','ltv','int_rt','channelB','channelC','channelR','channelT','ppmt_pnltyN','ppmt_pnltyX','ppmt_pnltyY','prop_typeCO','prop_typeCP','prop_typeLH','prop_typeMH','prop_typePU','prop_typeSF','prop_typeXX','loan_purposeC','loan_purposeN','loan_purposeP','orig_loan_term','cnt_borr']
    
    for x in cols_to_keep:
        if not x in df.columns:
            df[x] = 0.0
            
    df = df._get_numeric_data()
    df.drop('cd_msa',axis=1,inplace=True)
    df.drop('dt_first_pi',axis=1,inplace=True)
    df.drop('dt_matr',axis=1,inplace=True)
    df.drop('flag_sc',axis=1,inplace=True)
    df.drop('zipcode',axis=1,inplace=True)
    
    return df

In [11]:
def prepare_data_for_model(df):
    dummies1 = pd.get_dummies(df['flag_fthb']).rename(columns=lambda x: 'flag_fthb' + str(x))
    train_df = pd.concat([df, dummies1], axis=1)
    
    dummies2 = pd.get_dummies(df['occpy_sts']).rename(columns=lambda x: 'occpy_sts' + str(x)) 
    train_df = pd.concat([train_df, dummies2], axis=1)
    
    dummies3 = pd.get_dummies(df['channel']).rename(columns=lambda x: 'channel' + str(x)) 
    train_df = pd.concat([train_df, dummies3], axis=1)
    
    dummies4 = pd.get_dummies(df['ppmt_pnlty']).rename(columns=lambda x: 'ppmt_pnlty' + str(x)) 
    train_df = pd.concat([train_df, dummies4], axis=1)
    
    dummies5 = pd.get_dummies(df['prop_type']).rename(columns=lambda x: 'prop_type' + str(x)) 
    train_df = pd.concat([train_df, dummies5], axis=1)
    
    dummies6 = pd.get_dummies(df['loan_purpose']).rename(columns=lambda x: 'loan_purpose' + str(x)) 
    train_df = pd.concat([train_df, dummies6], axis=1)
    
    train_df['flag_sc'] = train_df['flag_sc'].map({'Y':1,'N':0})
    
    train_df = checkAllReqColumns(train_df)
    
    return train_df

In [12]:
def compute_metrics(model,predicted_val,true_val):
    RSq = r2_score(true_val,predicted_val)
    print('R Squared: ' + str(RSq))
    MAE = mean_absolute_error(true_val,predicted_val)
    print('MAE: ' + str(MAE))
    RMS = np.sqrt(mean_squared_error(true_val,predicted_val))
    print('RMS: ' + str(RMS))
    MAPE = np.mean(np.abs((true_val - predicted_val) / true_val)) * 100
    print('MAPE: ' + str(MAPE))
    print('\n')

## Financial Crisis :

In [13]:
first_qtr_str = input('Please Enter the first QuarterYear as QXYYYY: ')

Please Enter the first QuarterYear as QXYYYY: Q12005


In [14]:
second_qtr_str = get_next_quarter_str(first_qtr_str)
if int(second_qtr_str[2:]) not in list(range(2005,2017)):
        print("The Quarters requested are out of range")
        exit(0)
        
third_qtr_str = get_next_quarter_str(second_qtr_str)
if int(third_qtr_str[2:]) not in list(range(2005,2017)):
        print("The Quarters requested are out of range")
        exit(0)
        
fourth_qtr_str = get_next_quarter_str(third_qtr_str)
if int(fourth_qtr_str[2:]) not in list(range(2005,2017)):
        print("The Quarters requested are out of range")
        exit(0)

navigate()
get_4_quarter_data(first_qtr_str,second_qtr_str,third_qtr_str,fourth_qtr_str)

for file in file_names:
    extract_zipped_files_to_cwd(download_path + '\\' + file)

get_Origination_Quarter_File(folder_path_all_files + "/historical_data1_*.txt")

In [15]:
#Features Obtained from FWD and BWD Search
list_of_features = ['fico','mi_pct','cnt_units','orig_upb','ltv','orig_loan_term','cnt_borr','Year','flag_fthbN',
                   'flag_fthbY','occpy_stsI','occpy_stsP','occpy_stsS','channelT','ppmt_pnltyN','prop_typeCO',
                   'prop_typeCP','prop_typeMH','int_rt']

In [16]:
df1 = pd.read_csv(file_names[0].split('.')[0] + '.csv',low_memory=False)
df2 = pd.read_csv(file_names[1].split('.')[0] + '.csv',low_memory=False)
df3 = pd.read_csv(file_names[2].split('.')[0] + '.csv',low_memory=False)
df4 = pd.read_csv(file_names[3].split('.')[0] + '.csv',low_memory=False)

processed_df1 = prepare_data_for_model(df1)
processed_df2 = prepare_data_for_model(df2)
processed_df3 = prepare_data_for_model(df3)
processed_df4 = prepare_data_for_model(df4)

In [17]:
svr = LinearSVR(C=0.01, dual=True, epsilon=0.01, loss="squared_epsilon_insensitive", tol=1e-05)

In [18]:
svr.fit(processed_df1[list_of_features].drop('int_rt',axis=1),processed_df1['int_rt'])

LinearSVR(C=0.01, dual=True, epsilon=0.01, fit_intercept=True,
     intercept_scaling=1.0, loss='squared_epsilon_insensitive',
     max_iter=1000, random_state=None, tol=1e-05, verbose=0)

In [19]:
predicted1 = svr.predict(processed_df2[list_of_features].drop('int_rt',axis=1))
compute_metrics(svr,predicted1,processed_df2['int_rt'])

R Squared: -0.829132932203128
MAE: 0.3925374409746453
RMS: 0.47415649069235977
MAPE: 6.954675205285368




In [20]:
svr.fit(processed_df2[list_of_features].drop('int_rt',axis=1),processed_df2['int_rt'])

LinearSVR(C=0.01, dual=True, epsilon=0.01, fit_intercept=True,
     intercept_scaling=1.0, loss='squared_epsilon_insensitive',
     max_iter=1000, random_state=None, tol=1e-05, verbose=0)

In [21]:
predicted2 = svr.predict(processed_df3[list_of_features].drop('int_rt',axis=1))
compute_metrics(svr,predicted2,processed_df3['int_rt'])

R Squared: -9.541401518422159
MAE: 0.9284391769860116
RMS: 1.1072312131794202
MAPE: 16.107478137017644




In [22]:
svr.fit(processed_df3[list_of_features].drop('int_rt',axis=1),processed_df3['int_rt'])

LinearSVR(C=0.01, dual=True, epsilon=0.01, fit_intercept=True,
     intercept_scaling=1.0, loss='squared_epsilon_insensitive',
     max_iter=1000, random_state=None, tol=1e-05, verbose=0)

In [23]:
predicted3 = svr.predict(processed_df4[list_of_features].drop('int_rt',axis=1))
compute_metrics(svr,predicted3,processed_df4['int_rt'])

R Squared: -0.168102644176632
MAE: 0.3304790321780425
RMS: 0.4183467283827056
MAPE: 5.571970644563481




#### There is an increase in the Errors as we predict the next quarter interest rate from the rolling data of the previous years.

## Two Years Later :

In [24]:
first_qtr_str = 'Q12009'
second_qtr_str = get_next_quarter_str(first_qtr_str)
if int(second_qtr_str[2:]) not in list(range(2005,2017)):
        print("The Quarters requested are out of range")
        exit(0)
        
third_qtr_str = get_next_quarter_str(second_qtr_str)
if int(third_qtr_str[2:]) not in list(range(2005,2017)):
        print("The Quarters requested are out of range")
        exit(0)
        
fourth_qtr_str = get_next_quarter_str(third_qtr_str)
if int(fourth_qtr_str[2:]) not in list(range(2005,2017)):
        print("The Quarters requested are out of range")
        exit(0)

navigate()
get_4_quarter_data(first_qtr_str,second_qtr_str,third_qtr_str,fourth_qtr_str)

for file in file_names:
    extract_zipped_files_to_cwd(download_path + '\\' + file)

get_Origination_Quarter_File(folder_path_all_files + "/historical_data1_*.txt")

In [25]:
df1 = pd.read_csv(file_names[0].split('.')[0] + '.csv',low_memory=False)
df2 = pd.read_csv(file_names[1].split('.')[0] + '.csv',low_memory=False)
df3 = pd.read_csv(file_names[2].split('.')[0] + '.csv',low_memory=False)
df4 = pd.read_csv(file_names[3].split('.')[0] + '.csv',low_memory=False)


processed_df1 = prepare_data_for_model(df1)
processed_df2 = prepare_data_for_model(df2)
processed_df3 = prepare_data_for_model(df3)
processed_df4 = prepare_data_for_model(df4)

In [26]:
svr = LinearSVR(C=0.01, dual=True, epsilon=0.01, loss="squared_epsilon_insensitive", tol=1e-05)

In [27]:
svr.fit(processed_df1[list_of_features].drop('int_rt',axis=1),processed_df1['int_rt'])

LinearSVR(C=0.01, dual=True, epsilon=0.01, fit_intercept=True,
     intercept_scaling=1.0, loss='squared_epsilon_insensitive',
     max_iter=1000, random_state=None, tol=1e-05, verbose=0)

In [28]:
predicted1 = svr.predict(processed_df2[list_of_features].drop('int_rt',axis=1))
compute_metrics(svr,predicted1,processed_df2['int_rt'])

R Squared: -0.3559264812191798
MAE: 0.3372989255605931
RMS: 0.4006004665145859
MAPE: 7.09576531125427




In [29]:
svr.fit(processed_df2[list_of_features].drop('int_rt',axis=1),processed_df2['int_rt'])

LinearSVR(C=0.01, dual=True, epsilon=0.01, fit_intercept=True,
     intercept_scaling=1.0, loss='squared_epsilon_insensitive',
     max_iter=1000, random_state=None, tol=1e-05, verbose=0)

In [30]:
predicted2 = svr.predict(processed_df3[list_of_features].drop('int_rt',axis=1))
compute_metrics(svr,predicted2,processed_df3['int_rt'])

R Squared: -0.8523449627617261
MAE: 0.4505774209204374
RMS: 0.5615577238188115
MAPE: 8.511243447561862




In [31]:
svr.fit(processed_df3[list_of_features].drop('int_rt',axis=1),processed_df3['int_rt'])

LinearSVR(C=0.01, dual=True, epsilon=0.01, fit_intercept=True,
     intercept_scaling=1.0, loss='squared_epsilon_insensitive',
     max_iter=1000, random_state=None, tol=1e-05, verbose=0)

In [32]:
predicted3 = svr.predict(processed_df4[list_of_features].drop('int_rt',axis=1))
compute_metrics(svr,predicted3,processed_df4['int_rt'])

R Squared: -1.1269537027199874
MAE: 0.38420444523483277
RMS: 0.5289224513057039
MAPE: 7.71183360420543


